In [20]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

In [21]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [22]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [23]:
dim(test)
dim(train)

[1] 318 184

[1] 741 184

In [24]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    370     370 

## Model

In [25]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [26]:

model_svmPoly <- train(x=TrainData, y=TrainClasses, 
               method = "svmPoly", scale= FALSE) 

In [27]:

model_svmRadial <- train(x=TrainData, y=TrainClasses, 
               method = "svmRadial", scale= FALSE) 

Warning message in .local(x, ...):
"Variable(s) `' constant. Cannot scale data."


In [28]:

model_svmLinear <- train(x=TrainData, y=TrainClasses, 
               method = "svmLinear", scale= FALSE)

# Results

In [29]:
print(model_svmPoly)
plot(model_svmPoly,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Polynomial Kernel 

740 samples
183 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 740, 740, 740, 740, 740, 740, ... 
Resampling results across tuning parameters:

  degree  scale  C     Accuracy   Kappa    
  1       0.001  0.25  0.5868697  0.1951300
  1       0.001  0.50  0.7511516  0.5040208
  1       0.001  1.00  0.7764807  0.5532931
  1       0.010  0.25  0.8109626  0.6218795
  1       0.010  0.50  0.8308488  0.6615325
  1       0.010  1.00  0.8446682  0.6892042
  1       0.100  0.25  0.8537819  0.7074465
  1       0.100  0.50  0.8608809  0.7216262
  1       0.100  1.00  0.8695373  0.7388248
  2       0.001  0.25  0.7537848  0.5091641
  2       0.001  0.50  0.7795826  0.5594889
  2       0.001  1.00  0.8037357  0.6076652
  2       0.010  0.25  0.8422491  0.6844096
  2       0.010  0.50  0.8538683  0.7076388
  2       0.010  1.00  0.8651421  0.7300864
  2       0.100  0.25  0.90

In [30]:
print(model_svmRadial)
plot(model_svmRadial,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Radial Basis Function Kernel 

740 samples
183 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 740, 740, 740, 740, 740, 740, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.8832206  0.7660756
  0.50  0.8961058  0.7918298
  1.00  0.9083384  0.8163767

Tuning parameter 'sigma' was held constant at a value of 0.0222916
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.0222916 and C = 1.


In [31]:
print(model_svmLinear)
plot(model_svmLinear)

Support Vector Machines with Linear Kernel 

740 samples
183 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 740, 740, 740, 740, 740, 740, ... 
Resampling results:

  Accuracy   Kappa    
  0.8809166  0.7611698

Tuning parameter 'C' was held constant at a value of 1


ERROR: Error in plot.train(model_svmLinear): There are no tuning parameters with more than 1 value.


In [32]:
# collect resamples
results <- resamples(list(Poly=model_svmPoly,  Radial=model_svmRadial, Linear=model_svmLinear))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: Poly, Radial, Linear 
Number of resamples: 25 

Accuracy 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.8731343 0.8905660 0.9052632 0.9020058 0.9139785 0.9354839    0
Radial 0.8868613 0.8992806 0.9051095 0.9083384 0.9178571 0.9338235    0
Linear 0.8308824 0.8745645 0.8812261 0.8809166 0.8929889 0.9116608    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.7470575 0.7812224 0.8101828 0.8038563 0.8270572 0.8713693    0
Radial 0.7750411 0.7973121 0.8096612 0.8163767 0.8353110 0.8664120    0
Linear 0.6621300 0.7482211 0.7596663 0.7611698 0.7867705 0.8228903    0


# Prediction

In [33]:
pred_svmPoly <- predict(model_svmPoly,TestData)
pred_svmPoly.prob <- predict(model_svmPoly,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmPoly) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      20      22
   NotFast   46     230
                                         
               Accuracy : 0.7862         
                 95% CI : (0.737, 0.8299)
    No Information Rate : 0.7925         
    P-Value [Acc > NIR] : 0.639626       
                                         
                  Kappa : 0.2492         
                                         
 Mcnemar's Test P-Value : 0.005285       
                                         
            Sensitivity : 0.30303        
            Specificity : 0.91270        
         Pos Pred Value : 0.47619        
         Neg Pred Value : 0.83333        
             Prevalence : 0.20755        
         Detection Rate : 0.06289        
   Detection Prevalence : 0.13208        
      Balanced Accuracy : 0.60786        
                                         
       'Positive' Class : fast           
                                   

Sensitivity          Specificity       Pos Pred Value 
          0.30303030           0.91269841           0.47619048 
      Neg Pred Value            Precision               Recall 
          0.83333333           0.47619048           0.30303030 
                  F1           Prevalence       Detection Rate 
          0.37037037           0.20754717           0.06289308 
Detection Prevalence    Balanced Accuracy 
          0.13207547           0.60786436

In [34]:
pred_svmRadial <- predict(model_svmRadial,TestData)
pred_svmRadial.prob <- predict(model_svmRadial,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmRadial) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      22      20
   NotFast   42     234
                                          
               Accuracy : 0.805           
                 95% CI : (0.7571, 0.8471)
    No Information Rate : 0.7987          
    P-Value [Acc > NIR] : 0.422164        
                                          
                  Kappa : 0.3041          
                                          
 Mcnemar's Test P-Value : 0.007653        
                                          
            Sensitivity : 0.34375         
            Specificity : 0.92126         
         Pos Pred Value : 0.52381         
         Neg Pred Value : 0.84783         
             Prevalence : 0.20126         
         Detection Rate : 0.06918         
   Detection Prevalence : 0.13208         
      Balanced Accuracy : 0.63250         
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
          0.34375000           0.92125984           0.52380952 
      Neg Pred Value            Precision               Recall 
          0.84782609           0.52380952           0.34375000 
                  F1           Prevalence       Detection Rate 
          0.41509434           0.20125786           0.06918239 
Detection Prevalence    Balanced Accuracy 
          0.13207547           0.63250492

In [35]:
pred_svmLinear <- predict(model_svmLinear,TestData)
pred_svmLinear.prob <- predict(model_svmLinear,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmLinear) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      23      19
   NotFast   46     230
                                         
               Accuracy : 0.7956         
                 95% CI : (0.747, 0.8386)
    No Information Rate : 0.783          
    P-Value [Acc > NIR] : 0.32058        
                                         
                  Kappa : 0.2994         
                                         
 Mcnemar's Test P-Value : 0.00126        
                                         
            Sensitivity : 0.33333        
            Specificity : 0.92369        
         Pos Pred Value : 0.54762        
         Neg Pred Value : 0.83333        
             Prevalence : 0.21698        
         Detection Rate : 0.07233        
   Detection Prevalence : 0.13208        
      Balanced Accuracy : 0.62851        
                                         
       'Positive' Class : fast           
                                   

Sensitivity          Specificity       Pos Pred Value 
          0.33333333           0.92369478           0.54761905 
      Neg Pred Value            Precision               Recall 
          0.83333333           0.54761905           0.33333333 
                  F1           Prevalence       Detection Rate 
          0.41441441           0.21698113           0.07232704 
Detection Prevalence    Balanced Accuracy 
          0.13207547           0.62851406

In [36]:
roc_obj <- roc(TestClasses, as.numeric(pred_svmRadial))
plot(roc_obj, main="Curva ROC")


Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [37]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_svmRadial))

Data: as.numeric(pred_svmRadial) in 42 controls (TestClasses fast) < 276 cases (TestClasses NotFast).
Area under the curve: 0.6858

In [38]:
pred_svmRadial.prob

fast,NotFast
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
